# Hoja de Trabajo 2

## Integrantes

### Sergio Orellana - 221122

### Andre Marroquin - 22266

### Rodrigo Mansilla - 22611

# Link del repositorio

https://github.com/mar22266/LABORATORIOS-IA.git





# Task 1 - Preguntas Teóricas

Responda a cada de las siguientes preguntas de forma clara y lo más completamente posible.
1. Defina el proceso de decisión de Markov (MDP) y explique sus componentes.

        R//


2. Describa cual es la diferencia entre política, evaluación de políticas, mejora de políticas e iteración de políticas en el contexto de los PDM.

        R//



3. Explique el concepto de factor de descuento (gamma) en los MDP. ¿Cómo influye en la toma de decisiones?

        R//


4. Analice la diferencia entre los algoritmos de iteración de valores y de iteración de políticas para resolver MDP.

        R//


5. ¿Cuáles son algunos desafíos o limitaciones comunes asociados con la resolución de MDP a gran escala?
Discuta los enfoques potenciales para abordar estos desafíos.

        R//

# Task 2 - Preguntas Analíticas

Responda a cada de las siguientes preguntas de forma clara y lo más completamente posible.

1. Analice críticamente los supuestos subyacentes a la propiedad de Markov en los Procesos de Decisión de
Markov (MDP). Analice escenarios en los que estos supuestos puedan no ser válidos y sus implicaciones
para la toma de decisiones.

        R//


2. Explore los desafíos de modelar la incertidumbre en los procesos de decisión de Markov (MDP) y analice
estrategias para una toma de decisiones sólida en entornos inciertos.

        R//

# Task 3 - Preguntas Prácticas

In [ ]:
import numpy as np
import random

# Establecer semilla para reproducibilidad
seed = 42
random.seed(seed)
np.random.seed(seed)

# Dimension del entorno 4x4
N = 4

# Definir las posibles configuraciones de inicio y meta 
configuraciones = [
    ((0, 0), (N - 1, N - 1)),
    ((0, N - 1), (N - 1, 0)),
    ((N - 1, 0), (0, N - 1)),
    ((N - 1, N - 1), (0, 0))
]
start, goal = random.choice(configuraciones)

# Determinar aleatoriamente el numero de hoyos entre 0 y 3
n_holes = random.randint(0, 3)

# Seleccionar posiciones aleatorias para los hoyos 
posibles_posiciones = [(i, j) for i in range(N) for j in range(N) if (i, j) not in [start, goal]]
holes = random.sample(posibles_posiciones, n_holes)

# Crear el entorno
grid = [['F' for _ in range(N)] for _ in range(N)]
grid[start[0]][start[1]] = 'S'
grid[goal[0]][goal[1]] = 'G'
for h in holes:
    grid[h[0]][h[1]] = 'H'

# Mostrar el entorno generado
print("Entorno Frozen Lake:")
for row in grid:
    print(' '.join(row))

# Definicion de la dinamica del MDP
# 0: arriba, 1: derecha, 2: abajo, 3: izquierda
acciones = {
    0: (-1, 0),  
    1: (0, 1),   
    2: (1, 0),   
    3: (0, -1)   
}

# Retorna True si el estado es terminal meta o hoyo
def es_terminal(state):
    i, j = state
    return grid[i][j] in ['G', 'H']


# Dado un estado y una accion, retorna el siguiente estado. Si la accion lleva fuera de limites, se queda en el mismo estado. Si el estado ya es terminal, no se realiza movimiento.
def obtener_siguiente_estado(state, action):
    if es_terminal(state):
        return state
    di, dj = acciones[action]
    new_state = (state[0] + di, state[1] + dj)
    # Verificar limites del entorno
    if new_state[0] < 0 or new_state[0] >= N or new_state[1] < 0 or new_state[1] >= N:
        return state
    return new_state

# Retorna la recompensa de la transicion. Se muestra 1 si se llega a la meta, 0 en otro caso.
def obtener_recompensa(state, next_state):
    i, j = next_state
    if grid[i][j] == 'G':
        return 1.0
    else:
        return 0.0

# Algoritmo de Iteracion de Valores para resolver el MDP
# Parametros del MDP
gamma = 0.99   # Factor de descuento
theta = 1e-4   # Criterio de convergencia

# Inicializar la funcion de valor V(s) para cada estado en la grilla
V = np.zeros((N, N))

# Implementacion del algoritmo de iteracion de valores.
def value_iteration():
    while True:
        delta = 0
        # Iterar sobre cada celda 
        for i in range(N):
            for j in range(N):
                state = (i, j)
                if es_terminal(state):
                    continue
                v = V[i, j]
                # Calcular el valor de cada accion posible
                valores_acciones = []
                for a in acciones.keys():
                    next_state = obtener_siguiente_estado(state, a)
                    r = obtener_recompensa(state, next_state)
                    # Como la transicion es deterministica:
                    valores_acciones.append(r + gamma * V[next_state[0], next_state[1]])
                # Actualizar V(s) con el maximo valor
                V[i, j] = max(valores_acciones)
                delta = max(delta, abs(v - V[i, j]))
        # Verificar convergencia
        if delta < theta:
            break

# Extrae la politica optima a partir de la funcion de valor V. Para cada estado no terminal, se elige la accion que maximiza la recompensa futura.
def extraer_politica():
    policy = np.full((N, N), -1, dtype=int)  
    for i in range(N):
        for j in range(N):
            state = (i, j)
            if es_terminal(state):
                policy[i, j] = -1
            else:
                valores_acciones = {}
                for a in acciones.keys():
                    next_state = obtener_siguiente_estado(state, a)
                    r = obtener_recompensa(state, next_state)
                    valores_acciones[a] = r + gamma * V[next_state[0], next_state[1]]
                mejor_accion = max(valores_acciones, key=valores_acciones.get)
                policy[i, j] = mejor_accion
    return policy

# Ejecutar la iteracion de valores para calcular V(s)
value_iteration()
print("\nFuncion de valor V(s):")
print(V)

# Extraer la politica optima
policy = extraer_politica()

# Mapear acciones a simbolos para visualizacion
# 0: ↑, 1: →, 2: ↓, 3: ←, -1: T estado terminal
action_symbols = {0: '↑', 1: '→', 2: '↓', 3: '←', -1: 'T'}

print("\nPolitica optima:")
for i in range(N):
    row_symbols = []
    for j in range(N):
        # Mostrar S, G o H segun corresponda; de lo contrario, el simbolo de la accion
        if (i, j) == start:
            row_symbols.append('S')
        elif (i, j) == goal:
            row_symbols.append('G')
        elif grid[i][j] == 'H':
            row_symbols.append('H')
        else:
            row_symbols.append(action_symbols[policy[i, j]])
    print(' '.join(row_symbols))

# Funcion que encapsula la ejecucion del MDP y la visualizacion de resultados.
def ejecutar_simulacion():
    current_state = start
    trajectory = [current_state]
    # Simular trayecto siguiendo la politica optima hasta llegar a un estado terminal
    while not es_terminal(current_state):
        action = policy[current_state[0]][current_state[1]]
        next_state = obtener_siguiente_estado(current_state, action)
        trajectory.append(next_state)
        current_state = next_state
        # Evitar bucle infinito en caso de ciclos
        if len(trajectory) > 100:
            break
    print("\nTrayecto simulado:")
    for state in trajectory:
        print(state)
    print("\nSimulacion completada.")

# Ejecutar la simulacion
ejecutar_simulacion()


Entorno Frozen Lake:
S F F F
F F F F
F F F F
F F F G

Funcion de valor V(s):
[[0.95099005 0.96059601 0.970299   0.9801    ]
 [0.96059601 0.970299   0.9801     0.99      ]
 [0.970299   0.9801     0.99       1.        ]
 [0.9801     0.99       1.         0.        ]]

Politica optima:
S → → ↓
→ → → ↓
→ → → ↓
→ → → G

Trayecto simulado:
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(1, 3)
(2, 3)
(3, 3)

Simulacion completada.


Prompt final:

Ya con este entorno Frozen Lake en Python basado en un MDP, usando una matriz 4x4 con inicio y meta en esquinas opuestas seleccionadas aleatoriamente y hasta 3 hoyos ubicados aleatoriamente necesito que me ayudes a termines el trayecto simulado a partir de la politica optima porfavor.

Reflexión:

Para obtener el resultado deseado, se precisaron ajustes en el prompt, especificando claramente el formato y las modificaciones requeridas en el código. Con estas precisiones se logró que el código cumpliera con los objetivos sin hacer la totalidad del trabajo. Siempre siendo muy claros para ayudar y simplificar el resultado del prompt